In [38]:
import PythonFunctions as pf
import pandas as pd
import re
import numpy as np
import sys
from multiprocessing import Pool
import glob
print(sys.version)

3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]


# Generate descriptions of dilemmas

We use Python code to generate prompts for the LLM. The prompts include a demographic profile of a survey respondents and a description of the two dilemma that the survey respondents encountered during the Moral Machine experiment. 

The demographic profile includes the age, education, gender, and income that survey respondents reported in an optional demographic survey upon completion of the experiment. The order in which these characteristics appear in the prompt is randomized. 

The description of the scenarios is generated from the replication data provided by Awad et al. (2018). One dilemma consists of two scenarios, presented side by side during the experiment. 

To generate the descriptions of the dilemmas, we extended code written by Takemoto (2024). This study generates new dilemmas by randomly combining features of scenarios (e.g. the composition of characters) and prompts LLMs to evaluate these dilemmas. In contrast, our study takes the existing dilemmas from Awad et al. (2018) and prompts LLMs to predict how survey respondents evluated the dilemmas.

Below we illustrate how the entries in the data matrix describe a scenario. We created the images with the design functionalities of [moralmachine.net](https://www.moralmachine.net/). These images illustrate the examples but play no further role in the study. 

### Example 1

On the left side (`LeftHand=1`), respondents saw an AV that swerves to the other lane (`Intervention=1`) and kills 5 pedestrians (`Barrier=0`) – 1 baby, 1 female athlete, 1 male athlete, 1 female doctor, and 1 cat – who were crossing on a green light (`CrossingSignal=1`).

On the right side (`LeftHand=0`), respondents saw an AV that would continue ahead and crash into a barrier, resulting in the dealth of the 4 passengers (`Barrier=1`) - 1 baby, 1 female athlete, 1 female doctor, and a cat.

This example is taken from the [supplementary material](https://osf.io/wt6mc?view_only=4bb49492edee4a8eb1758552a362a2cf) in Awad et al. (2018). 
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/2224g4ytARX4QT5rB.png" alt="Example 1 2224g4ytARX4QT5rB (SI, Awad et al. 2018)" width="70%" align="center"/>
</div>

In [39]:
data1 = {
    "ResponseID": ["2224g4ytARX4QT5rB", "2224g4ytARX4QT5rB"],
    "ExtendedSessionID": ["213978760_9992828917431898.0", "213978760_9992828917431898.0"],
    "UserID": [9.992829e+15, 9.992829e+15],
    # Imputed demographics just for this illustration
    "Review_age": [36,36],                     
    "Review_education": ["bachelor","bachelor"],
    "Review_gender": ["Man","Man"],
    "Review_ContinuousIncome": [30000,30000],
    "Review_political": [54,54],
    "Review_religious": [24,24],
    "ScenarioOrder": [7, 7],
    "Intervention": [1, 0],
    "PedPed": [0, 0],
    "Barrier": [0, 1],
    "CrossingSignal": [1, 0],
    "AttributeLevel": ["More", "Less"],
    "ScenarioTypeStrict": ["Utilitarian", "Utilitarian"],
    "ScenarioType": ["Utilitarian", "Utilitarian"],
    "DefaultChoice": ["More", "More"],
    "NonDefaultChoice": ["Less", "Less"],
    "DefaultChoiceIsOmission": [0, 0],
    "NumberOfCharacters": [5, 4],
    "DiffNumberOFCharacters": [1, 1],
    "Saved": [0, 1],
    'Label': ['Case 2','Case 2'],
    "Template": ["Desktop", "Desktop"],
    "DescriptionShown": [1, 1],
    "LeftHand": [1, 0],
    "UserCountry3": ["USA", "USA"],
    "Man": [0, 0],
    "Woman": [0, 0],
    "Pregnant": [0, 0],
    "Stroller": [1, 1],
    "OldMan": [0, 0],
    "OldWoman": [0, 0],
    "Boy": [0, 0],
    "Girl": [0, 0],
    "Homeless": [0, 0],
    "LargeWoman": [0, 0],
    "LargeMan": [0, 0],
    "Criminal": [0, 0],
    "MaleExecutive": [0, 0],
    "FemaleExecutive": [0, 0],
    "FemaleAthlete": [1, 1],
    "MaleAthlete": [1, 0],
    "FemaleDoctor": [1, 1],
    "MaleDoctor": [0, 0],
    "Dog": [0, 0],
    "Cat": [1, 1]
}
df1 = pd.DataFrame(data1)
 

print("With persona:\n",pf.generate_scenario(df1, include_persona=True),"\n")
print("Label assigned by LLM: ",df1["Label"].unique())
print("Outcomes for these scenarios:\n",df1.apply(pf.classify_response, column_name = "Label", axis=1))

With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - You earn an annual income of 30,000 US dollars.\n - Your highest level of education is a bachelor degree.\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 24 for your religious views.\n - You are a man.\n - You are 36 years old.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 54 for your political views.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will swerve to avoid crashing into a concrete barrier and drive through a pedestrian crossing in the other lane. This will result in the death of 1 baby, 1 cat, 1 male athlete, 1 female doctor, and 1 female athle

### Example 2

These scenarios pit two groups of pedestrians against each other (`PedPed=1`).  

On the left side of the screen (`LeftHand=1`), respondents saw a scenario in which the AV stays on course (`Intervention=0`), resulting in the death of 1 man who was crossing on a red signal (`CrossingSignal=2`). 

On the right side of the screen (`LeftHand=0`), respondents saw a scenario in which the AV swerves to the other lane (`Intervention=1`), resulting in the death 1 male athlete who was crossing on a green signal (`CrossingSignal=1`).
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/22qKv8AmPcXEnNd8z.png" width="70%" align="center"/>
</div>

In [40]:
data2 = {
    "ExtendedSessionID": ["1055565952_8316216477776195.0", "1055565952_8316216477776195.0"],
    "ResponseID": ["22qKv8AmPcXEnNd8z", "22qKv8AmPcXEnNd8z"],
    "UserID": [8.316216e+15, 8.316216e+15],
    "Review_age": [29, 29],
    "Review_education": ["high","high"],
    "Review_income": ["10000", "10000"],
    "Review_gender": ["Man", "Man"],
    "Review_ContinuousIncome": [12500,12500],
    "IncomeBracketSmall": ["$5,001-\n$25,000", "$5,001-\n$25,000"],
    "Review_political": [100, 100],
    "Review_religious": [0, 0],
    "ScenarioOrder": [6, 6],
    "Intervention": [0, 1],
    "PedPed": [1, 1],
    "Barrier": [0, 0],
    "CrossingSignal": [2, 1],
    "AttributeLevel": ["Fat", "Fit"],
    "ScenarioTypeStrict": ["Fitness", "Fitness"],
    "ScenarioType": ["Fitness", "Fitness"],
    "DefaultChoice": ["Fit", "Fit"],
    "NonDefaultChoice": ["Fat", "Fat"],
    "DefaultChoiceIsOmission": [0, 0],
    "NumberOfCharacters": [1, 1],
    "DiffNumberOFCharacters": [0, 0],
    "Saved": [0, 1],
    'Label': ['Case 1','Case 1'],
    "Template": ["Desktop", "Desktop"],
    "DescriptionShown": [1, 1],
    "LeftHand": [1, 0],
    "UserCountry3": ["USA", "USA"],
    "Man": [1, 0],
    "Woman": [0, 0],
    "Pregnant": [0, 0],
    "Stroller": [0, 0],
    "OldMan": [0, 0],
    "OldWoman": [0, 0],
    "Boy": [0, 0],
    "Girl": [0, 0],
    "Homeless": [0, 0],
    "LargeWoman": [0, 0],
    "LargeMan": [0, 0],
    "Criminal": [0, 0],
    "MaleExecutive": [0, 0],
    "FemaleExecutive": [0, 0],
    "FemaleAthlete": [0, 0],
    "MaleAthlete": [0, 1],
    "FemaleDoctor": [0, 0],
    "MaleDoctor": [0, 0],
    "Dog": [0, 0],
    "Cat": [0, 0],
}

df2 = pd.DataFrame(data2)
 
print("With persona:\n",pf.generate_scenario(df2, include_persona=True),"\n")
print("Label assigned by LLM: ",df2["Label"].unique())
print("Outcomes for these scenarios:\n",df2.apply(pf.classify_response, column_name = "Label", axis=1))


With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - You earn an annual income of 12,500 US dollars.\n - Your highest level of education is a high school diploma.\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 0 for your religious views.\n - You are a man.\n - You are 29 years old.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 100 for your political views.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes, divided by an island. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will continue ahead and drive through a pedestrian crossing ahead. This will result in the death of 1 man, who was flouting the law by crossing on the red signal ahead of the car. Meanwh

### Example 3

This scenario pits pedestrians against pedestrians (`PedPed=1`). On the left side of the screen (`LeftHand=1`), respondents saw a scenario in which the AV would stay on course (`Intervention=0`), resulting in the death of 1 male executive who was crossing (`CrossingSignal=0`).

On the right side of the screen (`LeftSide=0`), respodents saw a scenario in which the AV would swerve (`Intervention=1`), resulting in the death of a 1 female executive who was crossing (`CrossingSignal=0`).
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/A6GmXsYKGxyivAFzu.png" width="70%" align="center"/>
</div>

In [41]:
data3 = {
    'ExtendedSessionID': ['1694978322_3759038854820315.0', '1694978322_3759038854820315.0'],
    'ResponseID': ['A6GmXsYKGxyivAFzu', 'A6GmXsYKGxyivAFzu'],
    'UserID': [3.759039e+15, 3.759039e+15],
    'Review_age': [46, 46],
    'Review_education': ['bachelor','bachelor'],
    'Review_gender': ['Woman', 'Woman'],
    'Review_income': ['35000', '35000'],
    "Review_ContinuousIncome": [42500,42500],
    'IncomeBracketSmall': ['$25,001-\n$50,000', '$25,001-\n$50,000'],
    'Review_political': [11, 11],
    'Review_religious': [46, 46],
    'ScenarioOrder': [1, 1],
    'Intervention': [0, 1],
    'PedPed': [1, 1],
    'Barrier': [0, 0],
    'CrossingSignal': [0, 0],
    'AttributeLevel': ['Male', 'Female'],
    'ScenarioTypeStrict': ['Gender', 'Gender'],
    'ScenarioType': ['Gender', 'Gender'],
    'DefaultChoice': ['Male', 'Female'],
    'NonDefaultChoice': ['Male', 'Female'],
    'DefaultChoiceIsOmission': [1, 1],
    'NumberOfCharacters': [1, 1],
    'DiffNumberOFCharacters': [0, 0],
    'Saved': [0, 1],
    'Label': ['Case 1','Case 1'],
    'Template': ['Desktop', 'Desktop'],
    'DescriptionShown': [0, 0],
    'LeftHand': [1, 0],
    'UserCountry3': ['USA', 'USA'],
    'Man': [0, 0],
    'Woman': [0, 0],
    'Pregnant': [0, 0],
    'Stroller': [0, 0],
    'OldMan': [0, 0],
    'OldWoman': [0, 0],
    'Boy': [0, 0],
    'Girl': [0, 0],
    'Homeless': [0, 0],
    'LargeWoman': [0, 0],
    'LargeMan': [0, 0],
    'Criminal': [0, 0],
    'MaleExecutive': [1, 0],
    'FemaleExecutive': [0, 1],
    'FemaleAthlete': [0, 0],
    'MaleAthlete': [0, 0],
    'FemaleDoctor': [0, 0],
    'MaleDoctor': [0, 0],
    'Dog': [0, 0],
    'Cat': [0, 0]
}

df3 = pd.DataFrame(data3)

print("With persona:\n",pf.generate_scenario(df3, include_persona=True),"\n")
print("Label assigned by LLM: ",df3["Label"].unique())
print("Outcomes for these scenarios:\n",df3.apply(pf.classify_response, column_name = "Label", axis=1))

With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 46 for your religious views.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 11 for your political views.\n - You earn an annual income of 42,500 US dollars.\n - Your highest level of education is a bachelor degree.\n - You are 46 years old.\n - You are a woman.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes, divided by an island. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will continue ahead and drive through a pedestrian crossing ahead. This will result in the death of 1 male executive, who was crossing ahead of the car. Meanwhile, it will spare 1 female e

### Example 4

On the left side of the screen (`LeftHand=1`), respondents saw a scenario in which an AV would stay on course (`Intervention=0`), resulting in the death of 5 pedestrians (`Barrier=0`) – 1 man, 1 woman, 2 boys, and 1 girl – who were crossing on a green light (`CrossingSignal=1`). 

On the right side of the screen (`LeftHand=0`), respondents saw a scenario in which an AV would swerve onto the other lane (`Intervention=1`), resulting in the death of the 5 passengers (`Barrier=1`) – 1 man, 1 woman, 2 old men, and 1 old woman. 
<div style="text-align: center;">
    <img src="../Figures/4_ScenarioExamples/EH3SfatQP3hygSpzF.png" width="70%" align="center"/>
</div>

In [42]:
data4 = {
    'ExtendedSessionID': ['-2127483756_5144602155778557.0', '-2127483756_5144602155778557.0'],
    'ResponseID': ['EH3SfatQP3hygSpzF', 'EH3SfatQP3hygSpzF'],
    'UserID': [5.144602e+15, 5.144602e+15],
    'Review_gender': ['Man', 'Man'],
    'Review_age': [35, 35],
    'Review_ageBracket': ['35-44','35-44'],
    'Review_income': ['under5000', 'under5000'],
    'Review_ContinuousIncome': [2500,2500],
    'IncomeBracketSmall': ['$0-$5,000', '$0-$5,000'],
    'Review_education': ['high','high'],
    'Review_educationBracket': ['High school','High school'],
    'Review_political': [100, 100],
    'Review_religious': [0, 0],
    'ScenarioOrder': [3, 3],
    'Intervention': [0, 1],
    'PedPed': [0, 0],
    'Barrier': [0, 1],
    'CrossingSignal': [1, 0],
    'AttributeLevel': ['Young', 'Old'],
    'ScenarioTypeStrict': ['Age', 'Age'],
    'ScenarioType': ['Age', 'Age'],
    'DefaultChoice': ['Young', 'Young'],
    'NonDefaultChoice': ['Old', 'Old'],
    'DefaultChoiceIsOmission': [1, 1],
    'NumberOfCharacters': [5, 5],
    'DiffNumberOFCharacters': [0, 0],
    'Saved': [0, 1],
    'Label': ['Case 1','Case 1'],
    'Template': ['Mobile', 'Mobile'],
    'DescriptionShown': [0, 0],
    'LeftHand': [1, 0],
    'UserCountry3': ['USA', 'USA'],
    'Man': [1, 1],
    'Woman': [1, 1],
    'Pregnant': [0, 0],
    'Stroller': [0, 0],
    'OldMan': [0, 2],
    'OldWoman': [0, 1],
    'Boy': [2, 0],
    'Girl': [1, 0],
    'Homeless': [0, 0],
    'LargeWoman': [0, 0],
    'LargeMan': [0, 0],
    'Criminal': [0, 0],
    'MaleExecutive': [0, 0],
    'FemaleExecutive': [0, 0],
    'FemaleAthlete': [0, 0],
    'MaleAthlete': [0, 0],
    'FemaleDoctor': [0, 0],
    'MaleDoctor': [0, 0],
    'Dog': [0, 0],
    'Cat': [0, 0],
}

df4 = pd.DataFrame(data4)

print("With persona:\n",pf.generate_scenario(df4, include_persona=True),"\n")
print("Label assigned by LLM: ",df4["Label"].unique())
print("Outcomes for these scenarios:\n",df4.apply(pf.classify_response, column_name = "Label", axis=1))

With persona:
 ("Carefully consider the following information and think step by step. You are an American with the following characteristics:\n - On a scale from 0 (Not Religious) to 100 (Very Religious), you report a score of 0 for your religious views.\n - On a scale from 0 (Conservative) to 100 (Progressive), you report a score of 100 for your political views.\n - You earn an annual income of under 5,000 US dollars.\n - Your highest level of education is a high school diploma.\n - You are 35 years old.\n - You are a man.\nConsider that pedestrians are crossing a zebra crossing that spans two lanes. Please indicate which case is better for autonomous driving by choosing either 'Case 1' or 'Case 2'. Respond with one word only.\n", 'Case 1.\nA self-driving car with sudden brake failure will continue ahead and drive through a pedestrian crossing ahead. This will result in the death of 1 man, 1 girl, 2 boys, and 1 woman, who were abiding by the law by crossing on the green signal. Meanwh

# Prompt Language Models

In [ ]:
pd.set_option('display.max_columns', None)

# load survey data
mms_all = pd.read_csv("../Data/1_SurveySample.csv.gz")

# OpenAI API key  
oai_api_key = ""

print("Number of rows:", mms_all.shape[0],
      "\nNumber of prompts:", mms_all.shape[0]/2,
      "\nNumber of columns:", mms_all.shape[1],"\n")

# structure of dataset
print(pd.Series({c: mms_all[c].unique() for c in mms_all}))

In [ ]:
print("Number of missing values for each variable:")

print("Saved: ", mms_all["Saved"].isna().sum())
print("Education:", mms_all["Review_education"].isna().sum())
print("Gender:", mms_all["Review_gender"].isna().sum())
print("Income: ",  mms_all["Review_ContinuousIncome"].isna().sum())
print("Age:", mms_all["Review_age"].isna().sum())
print("Pol:", mms_all["Review_political"].isna().sum())
print("Rel:", mms_all["Review_religious"].isna().sum())


In the following, we split up the datasets into multiple chunks. This allows us to prompt using parallel processing.

In [ ]:
remaining = mms_all
remaining = remaining.reset_index(drop=True)

In [ ]:
# Split the IDs into 8 chunks
id_chunks = np.array_split(remaining["ResponseID"].unique(), 8)

# Create data chunks based on the ID chunks
data_chunks = [remaining[remaining["ResponseID"].isin(id_chunk)].copy() for id_chunk in id_chunks]

# Loop through each data chunk and save to a compressed CSV file
for i, chunk in enumerate(data_chunks, start=1):
    # Define the filename with the current index
    filename = f"../Data/3_gpt4turbo_wp_20241118_{i}.csv.gz"
    
    # Save the chunk to a compressed CSV file (only once)
    #chunk.to_csv(filename, compression='gzip', index=False)

    nids = len(chunk["ResponseID"].unique())
    
    print(f"Saved chunk {i} to {filename} with {nids} rows")


Next, we prompt the OpenAI API using parallel processing.

In [ ]:
# Assuming mms_all, oai_api_key, etc., are already defined
if __name__ == '__main__':
    
    # Number of processes to run in parallel
    num_processes = 8  

    # Choose the list of files to process ending on 1-8
    file_list = glob.glob("../Data/3_gpt4turbo_wp_20241118_[1-8].csv.gz")
    print(file_list)

    # Define the pattern to extract the chunk number from the filename
    pattern = re.compile(r'_(\d+)\.csv\.gz$')

    # Define the arguments for each chunk
    args_list = [
        (
            pd.read_csv(file),                                                            # data
            "gpt-4-turbo",                                                                # model
            oai_api_key,                                                                  # api_key
            f"../Data/3_gpt4turbo_wp_20241118_{pattern.search(file).group(1)}r.csv.gz",   # csv_path
            True,                                                                         # include_persona
            False,                                                                        # verbose
            0,                                                                          # sleep
            None                                                                          # temperature
        )
        for file in file_list
    ]

for index in range(len(args_list)):

    # Use mutiprocessing Pool to run prompt_llm in parallel
    with Pool(processes=num_processes) as pool:
        pool.starmap(pf.prompt_llm, args_list)

In [ ]:
# Get list of all files generated by processes
file_list = glob.glob("../Data/3_gpt4turbo_wp_20241118_[0-9]r.csv.gz")
print(file_list)

# Read and concatenate them
df_list = [pd.read_csv(file) for file in file_list]
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined dataframe
combined_df.to_csv("../Data/3_gpt4turbo_wp_20241118.csv.gz", compression="gzip", index=False)
combined_df

In [ ]:
# Check if all ResponseIDs in mms_all are in combined_df
missing_in_combined = mms_all[~mms_all['ResponseID'].isin(combined_df['ResponseID'])]

if missing_in_combined.empty:
    print("All ResponseIDs in mms_all are present in combined_df.")
else:
    print("The following ResponseIDs in mms_all are missing in combined_df:")
    print(missing_in_combined['ResponseID'].unique())

# Check if all ResponseIDs in combined_df are in mms_all
missing_in_mms_all = combined_df[~combined_df['ResponseID'].isin(mms_all['ResponseID'])]

if missing_in_mms_all.empty:
    print("All ResponseIDs in combined_df are present in mms_all.")
else:
    print("The following ResponseIDs in combined_df are missing in mms_all:")
    print(missing_in_mms_all['ResponseID'].unique())

In [ ]:
# gpt-3.5-turbo-0125
pf.prompt_llm(mms_all,model="gpt-3.5-turbo-0125", 
              csv_path="../Data/3_gpt35turbo0125_wp_20240603.csv.gz", 
              api_key=oai_api_key, 
              include_persona=True)

In [ ]:
# gpt-4o
pf.prompt_llm(mms_all,model="gpt-4o", 
              csv_path="../Data/3_gpt4o_wp_20240603.csv.gz", 
              api_key=oai_api_key, 
              include_persona=True)